# Siemens Star

The [Siemens Star](https://en.wikipedia.org/wiki/Siemens_star) is a radial starburst pattern commonly used to adjust internal focus settings in camera lens and for measuring camera resolution. This notebook produces a Siemens Star test chart in the form in [Scalable Vector Graphics (svg)[(https://en.wikipedia.org/wiki/Scalable_Vector_Graphics) that can be rendered by most modern web browsers.  The idea for this notebook was motivated by this [post](http://www.dpreview.com/forums/post/57422062) on a forum at [Digital Photography Review (DPReview)](http://www.dpreview.com/).

### Usage

This is a [Jupyter notebook](http://jupyter.org/) containing Python functions. An executable version of this notebook is available from the [http://mybinder.org/](http://mybinder.org/) by clicking on the following button.  There may be a longish pause as the notebook loads.

[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/jckantor/Siemens-Star)

To use the notebook after it has been loaded in an executable environment:

1. First run the code in the following cell. Do this by clicking in the cell to highlight it, then entering 'shift-return' from the keyboard, or clicking on the run button in the toolbar menu (looks like >|), or the 'Cell>Run Cells' menu item.
2. Execute the second cell below to create the desired chart interactively. The controls adjust the colors and number of spokes displayed in the chart.

### Function Definitions

The following cell defines functions that create SVG markup. Run this cell first!

In [1]:
from math import pi, sin, cos

# bounding box
bb_width = 900
aspect = 3.0/2.0
bb_height = bb_width/aspect

# returns svg string of polygons for one star a
def star(xc,yc,radius,cycles=72,colors=['black','white'],clines='True'): 
    hstep = pi/cycles/len(colors)
    polygon = '<polygon points="{0:.3f},{1:.3f}'.format(xc,yc)
    polygon += ' {0:.3f},{1:.3f}'.format(xc - radius*sin(hstep),yc + radius*cos(hstep))
    polygon += ' {0:.3f},{1:.3f}'.format(xc + radius*sin(hstep),yc + radius*cos(hstep)) + '"'
    str = ''
    for n in range(0,cycles):
        for k in range(0,len(colors)):
            str += polygon + ' style="fill:{0}"'.format(colors[k])
            str += ' transform="rotate({0:.3f},{1:.3f},{2:.3f})"/>\n'\
                .format(360.0*(n+float(k)/len(colors))/cycles,xc,yc)
    if clines:
        nlines = 0
        for lph in (10,20,50,100,200,500,1000,2000,5000):
            r = bb_height*cycles*len(colors)/(2*pi*lph)
            if (r < radius) & (nlines < 4):
                nlines += 1
                str += '<circle cx="{0}" cy="{1}" r="{2}"'.format(xc,yc,r)
                str += ' stroke="black" stroke-width="1" fill-opacity="0"/>\n'
                str += '<text x="{0}" y="{1}"'.format(xc + r*cos(13*pi/8),yc + r*sin(13*pi/8))
                str += ' fill="blue" font-weight="bold" text-align="center">'
                str += '{0:4.0f}</text>\n'.format(lph)
    return str

def colorcheck():
    rgb = [[115, 82, 68],[194,150,130],[ 98,122,157],[ 87,108, 67],[133,128,177],[103,189,170], \
           [214,126, 44],[ 80, 91,166],[193, 90, 99],[ 94, 60,108],[157,188, 64],[224,163, 46], \
           [ 56, 61,150],[ 70,148, 73],[175, 54, 60],[231,199, 31],[187, 86,149],[  8,133,161], \
           [243,243,242],[200,200,200],[160,160,160],[122,122,121],[ 85, 85, 85],[ 52, 52, 52]]
    bw = 0.05*bb_width
    bh = 0.05*bb_height
    rlist = ['<rect width="{0}" height="{1}" '.format(bw,bh) + 
             ' x="{0}" y="{1}"'.format(bw*(1+col) + (col>1)*14*bw, bh*(7+row)) + \
             ' style="fill:rgb({0},{1},{2});stroke-width:2;stroke:black"/>'.format(*rgb[row+6*col]) \
             for col in range(0,4) for row in range (0,6)]
    return '\n'.join(rlist) + '\n'

def SiemensStar(cycles=144, colors=['black','white'], ccheck='True', cstars='True', clines='True'):
    svg = '<svg xmlns="https://www.w3.org/2000/svg"' \
          + ' width="{0}" height="{1}" style="border:solid 1px" >\n'.format(bb_width,bb_height)   
    svg += star(0.50*bb_width, 0.5*bb_height, 0.48*bb_height, cycles, colors, clines) 
    if cstars:
        r = 0.15*bb_height
        for xloc in (r, bb_width - r):
            for yloc in (r, bb_height - r):
                svg += star(xloc, yloc, r, cycles/2, colors, clines) 
    if ccheck:
        svg += colorcheck()
    svg += '</svg>'
    return svg

### Interactively Create and Display Siemens Star

The following interactive cell creates a simple resolution chart consisting of a central Siemens Star and additional elements. Use the dropdown menus to set the colors of the radial spokes, and the cycles slider to set the number of spokes.

A list of svg commands is generated and stored in a file [star.html](star.html). To download the resulting file, right click on the link and choose the `Save Link As ...` option.

In [2]:
from IPython.display import SVG
from ipywidgets import interact, Dropdown, IntSlider, Checkbox
from IPython.display import display

svg_colors = ['black','white','red','blue','green','cyan','yellow','magenta']

def makestar(cycles,color1,color2,ccheck,cstars,clines):
    svg = SiemensStar(cycles, colors=[color1,color2], ccheck=ccheck, cstars=cstars, clines=clines)
    display(SVG(svg))
    with open('star.html','w') as f:
        f.write(svg)
    
interact(makestar, 
         cycles = IntSlider(min=12, max=480, value = 144, description='Color Cycles'), \
         color1 = Dropdown(options=svg_colors, value='black', description='Color A'), \
         color2 = Dropdown(options=svg_colors, value='white', description='Color B'), \
         ccheck = Checkbox(value=True, description='Color Checker'), \
         cstars = Checkbox(value=True, description='Corner Stars'), \
         clines = Checkbox(value=True, description='LPH Lines'))

<function __main__.makestar>

### Manually Create and Display the Siemens Star

The following cell creates a simple resolution chart consisting of a central Siemens Star and additional elements. The central star consists of alternating radial spokes of color from a specified list `colors`. If omitted, the default is black and white. The list may contain two or more colors. The list of colors is repeated a number of times as determined by the parameter `cycles`.

A list of svg commands is generated, stored in a file [star.html](star.html), and displayed as an inline graphic within the notebook. To download the resulting file, right click on the link and choose the `Save Link As ...` option.

In [3]:
from IPython.display import SVG

svg = SiemensStar(cycles=144,colors=['blue','yellow'])

f = open('star.html','w')
f.write(svg)
f.close()

SVG(svg)